In [4]:
import numpy as np
import pandas as pd
from tpot import TPOTClassifier

from dask.distributed import Client
import joblib

In [5]:
def get_df_from_default():
    df_temp = pd.read_csv("data/base.csv", sep=';')

    df_temp = df_temp[df_temp['Curso'] == 'Biologia']

    df = df_temp[['VAR01', 'VAR02', 'VAR03', 'VAR04', 'VAR05',
                  'VAR06', 'VAR07', 'VAR08', 'VAR09', 'VAR10',
                  'VAR12', 'VAR14', 'VAR15', 'VAR16', 'VAR17',
                  'VAR18', 'VAR19', 'VAR20', 'VAR21', 'VAR22',
                  'VAR23', 'VAR24', 'VAR25', 'VAR28', 'VAR30',
                  'VAR31', 'VAR31b', 'VAR31c', 'VAR32a', 'VAR32b',
                  'VAR32c', 'VAR32d', 'VAR33', 'VAR34', 'VAR35', 'DESEMPENHO_BINARIO']]

    df = df.rename(columns={"DESEMPENHO_BINARIO": "Y"})

    df["VAR17"] = df["VAR17"].astype(str).replace(',','.')
    df["VAR24"] = df["VAR24"].astype(str).replace(',','.')
    df["VAR25"] = df["VAR25"].astype(str).replace(',','.')
    df["VAR30"] = df["VAR30"].astype(str).replace(',','.')

    df["VAR17"] = df["VAR17"].astype(float)
    df["VAR24"] = df["VAR24"].astype(float)
    df["VAR25"] = df["VAR25"].astype(float)
    df["VAR30"] = df["VAR30"].astype(float)
    
    return df

In [65]:
def get_df_from_fmdev():
    df = pd.read_csv("../backend/data/raw/94ca2547-88c4-4c10-9446-c71a09dc579e.csv")
    
#     df_categoric = df.copy()
#     df_categoric = df_categoric.select_dtypes(include=['object'])
#     df = df.drop(df_categoric.columns, axis=1)
#     del df['id_do_aluno']

        
    
#     df = df[['var01', 'var02', 'var03', 'var04', 'var05',
#                   'var06', 'var07', 'var08', 'var09', 'var10',
#                   'var12', 'var14', 'var15', 'var16', 'var17',
#                   'var18', 'var19', 'var20', 'var21', 'var22',
#                   'var23', 'var24', 'var25', 'var28', 'var30',
#                   'var31', 'var31b', 'var31c', 'var32a', 'var32b',
#                   'var32c', 'var32d', 'var33', 'var34', 'var35', 'desempenho_binario']]

#     df = df.rename(columns={"desempenho_binario": "y"})

#     df["var17"] = df["var17"].astype(str).replace(',','.')
#     df["var24"] = df["var24"].astype(str).replace(',','.')
#     df["var25"] = df["var25"].astype(str).replace(',','.')
#     df["var30"] = df["var30"].astype(str).replace(',','.')

#     df["var17"] = df["var17"].astype(float)
#     df["var24"] = df["var24"].astype(float)
#     df["var25"] = df["var25"].astype(float)
#     df["var30"] = df["var30"].astype(float)
    
    return df

In [66]:
def get_split(df, target):
    msk = np.random.rand(len(df)) < 0.7

    train = df[msk]
    test = df[~msk]

    Y_test = test[target]
    X_test = test.drop(labels = [target],axis = 1)

    Y_train = train[target]
    X_train = train.drop(labels = [target],axis = 1)
    
    return X_train, Y_train, X_test, Y_test

In [67]:
df = get_df_from_fmdev()
X_train, Y_train, X_test, Y_test = get_split(df, 'desempenho_binario')

In [68]:
# connect to the cluster
client = Client(processes=False) 

# create the estimator normally
estimator = TPOTClassifier(generations=5, max_time_mins=5, population_size=20, cv=5,
                                    random_state=42, verbosity=3, 
                                    use_dask=True,
                                    periodic_checkpoint_folder="data/tpot-output/")

# perform the fit in this context manager
with joblib.parallel_backend("dask"):
    estimator.fit(X_train, Y_train)

31 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
Generation 1 - Current Pareto front scores:
-1	0.8834953012311502	RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.45, RandomForestClassifier__min_samples_leaf=9, RandomForestClassifier__min_samples_split=3, RandomForestClassifier__n_estimators=100)

Saving periodic pipeline from pareto front to data/tpot-output/pipeline_gen_1_idx_0_2020.04.10_21-13-17.py
_pre_test decorator: _random_mutation_operator: num_test=0 '(slice(None, None, None), 0)' is an invalid key.
Generation 2 - Current Pareto front scores:
-1	0.891593210461135	GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate=0.1, GradientBoostingClassifier

In [69]:
estimator.score(X_test, Y_test)

0.8987012987012987

In [62]:
for item in estimator.pareto_front_fitted_pipelines_:
    print(item.feature_importances_)

AttributeError: 'str' object has no attribute 'feature_importances_'

In [64]:
estimator.pareto_front_fitted_pipelines_.feature_importances_

KeyError: 0